In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


# Conversion a InfluxDB Annotated CSV

In [6]:
# Leer el dataset con limite de filas
# ===================================
dataset = pd.read_csv('EBW-Sample_data_Zylk_2022-07-01_0h_a_2023-02-09_0h.csv', sep=';', nrows=5000000)
dataset.shape

/tmp/ipykernel_24988/1952779246.py:3: DtypeWarning: Columns (2,3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../EBW-Sample_data_Zylk_2022-07-01_0h_a_2023-02-09_0h.csv', sep=';', nrows=5000000)


(5000000, 33)

In [7]:
# Convertir a formato dateTime:RFC3339 para InfluxDB
# ==================================================
dataset['_DateTimeLocal'] = pd.to_datetime(dataset['_DateTimeLocal'])
dataset['_DateTimeLocal'] = dataset['_DateTimeLocal'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

In [8]:
pd.set_option('display.max_columns', None)
dataset.head(2)

,id,_DateTimeLocal,OF,PN,status,PROG,BLOCK,BLOCK_S,AAVACTBX,AAVACTBY,ALIG_X,ALIG_Y,BEAN_CURR_ACT,BEAN_CURR_CMD,BIAS_VOLT_ACT,DEFL_FREC,DEFL_X,DEFL_Y,HIGH_VOLT_ACT,HIGH_VOLT_CMD,LENS_CURR_ACT,LENS_CURR_CMD,STIG_X,STIG_Y,TEMP_AGUA_OK,TEMP_DIF_ACT,TIEMP_PARA_VAC,VAC_CAM_ACT,VAC_CAM_CMD,VAC_CAN_ACT,VAC_CAN_CMD,VAC_CAN_ACT_2,VAC_CAM_ACT_2
0,58474487,2022-07-01T00:00:00.467000Z,222346055-0210-0,12008285.0,404,_N_MPF210_MPF,N0135,M00 ;(INTRODUCE ORDEN DE FABRICACION EN LA G...,0.0,0.0,50037.0,61854.0,5.0,0.0,24575.0,29.0,0.0,0.0,24871.0,27648.0,18120.0,13353.0,65343.0,63656.0,True,2313.0,0.0,23716.0,23716.0,9770.0,9770.0,9969.0,24696.0
1,58474488,2022-07-01T00:00:00.920000Z,222346055-0210-0,12008285.0,404,_N_MPF210_MPF,N0135,M00 ;(INTRODUCE ORDEN DE FABRICACION EN LA G...,0.0,0.0,50037.0,61854.0,7.0,0.0,24576.0,29.0,0.0,0.0,24872.0,27648.0,18120.0,13353.0,65343.0,63656.0,True,2313.0,0.0,23717.0,23717.0,9779.0,9779.0,9979.0,24696.0


# Transformar a Flux Annotated CSV

In [12]:
column_numericas= dataset[['AAVACTBX', 'AAVACTBY', 'BEAN_CURR_ACT',
       'BEAN_CURR_CMD', 'BIAS_VOLT_ACT', 
       'HIGH_VOLT_ACT', 'HIGH_VOLT_CMD', 'LENS_CURR_ACT', 'LENS_CURR_CMD',
       'TEMP_DIF_ACT', 'TIEMP_PARA_VAC',
       'VAC_CAM_ACT', 'VAC_CAM_CMD', 'VAC_CAN_ACT', 'VAC_CAN_CMD',
       'VAC_CAN_ACT_2', 'VAC_CAM_ACT_2']]

In [10]:
# Cargar CSV con las anotaciones
# ==============================

annotation = pd.read_csv('CSV_influxdb/influxdb-annotation.csv', sep=',')
annotation.head()

,#datatype,string,long,dateTime:RFC3339,string.1,double,string.2
0,#group,False,False,False,True,False,True
1,#default,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
for column in column_numericas:
   
    # Seleccionar _DateTimeLocal->_time ; PROG-> _field ; column->_value
    # ==================================================================
    influxdb = dataset.loc[:, ['_DateTimeLocal','PROG',column]]
    
    # Añadir columnas que hacen falta para Influx Annotated CSV
    # =========================================================
    influxdb.insert(0, '', '')
    influxdb.insert(1, 'result', '0')
    influxdb.insert(2, 'table', '0')
    influxdb['_measurement'] = column
    
    # Cambiar nombres a las columnas
    # ==============================
    influxdb = influxdb.rename(columns={'_DateTimeLocal': '_time', 'PROG': '_field', column: '_value'})
    
    influxdb.to_csv(f'CSV_influxdb/{column}.csv', index=False)
    
    with open('CSV_influxdb/influxdb-annotation.csv', 'r') as archivo1, open(f'CSV_influxdb/{column}.csv', 'r') as archivo2:
        contenido1 = archivo1.readlines()
        contenido2 = archivo2.readlines()

    contenido_completo = contenido1 + contenido2
    
    with open(f'CSV_influxdb/{column}.csv', 'w') as archivo_resultado:
        archivo_resultado.writelines(contenido_completo)
